In [99]:

%%HTML
<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>

In [100]:
# CELL 1
from astropy.utils import iers
#iers.conf.auto_download = False 
from astroplan import Observer
from astropy.coordinates import SkyCoord,EarthLocation,CartesianRepresentation
from astropy.time import Time
from astropy import units as u
from astropy.utils import iers
import datetime
import emoji
from IPython.display import HTML, display, clear_output, Javascript
from IPython.display import Image
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, Layout, Button, Box, HTML, TwoByTwoLayout, GridspecLayout
import matplotlib.pyplot as plt
import markdown 
import numpy as np
import os
import pandas as pd
from plotly import graph_objs as go
from colorama import Fore
from SPOCK import target_list_from_stargate_path
import SPOCK.stats as SPOCKstats
import SPOCK.long_term_scheduler as SPOCKLT
import SPOCK.short_term_scheduler as SPOCKST
import SPOCK.plots_scheduler as SPOCKplot
from SPOCK import target_list_from_stargate_path
import sys
import yaml

import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_rows = 2000
pd.options.display.max_columns = 80

iers.IERS_A_URL  ='ftp://cddis.gsfc.nasa.gov/pub/products/iers/finals2000A.all'

%matplotlib notebook


#from astroplan import download_IERS_A
#download_IERS_A

#check_user = widgets.Text(description='Username',style=style)
#check_password = widgets.Password(description='Password',style=style)

#credentials = widgets.HBox([check_user,check_password],layout=widgets.Layout(height='40px'))
#display(credentials)



In [101]:
# CELL 2




style = {'description_width':'initial'}

observatory_widget = widgets.Combobox(
    # value='John',
    placeholder='Choose site',
    options=['SSO', 'SNO', 'Saint-Ex', 'TS_La_Silla','TN_Oukaimeden'],
    description='Site:',
    ensure_option=True,
    disabled=False)

telescope_widget = widgets.Combobox(
    # value='John',
    placeholder='SSO telescope',
    options=['Io','Europa','Ganymede','Callisto'],
    description='Telescope:',
    ensure_option=True,
    disabled=False)

operator_widget = widgets.Combobox(
    # value='John',
    placeholder='Opertor for TS/TN',
    options=['Manu','Youssef','Fran','Khalid','Marin'],
    description='Operator:',
    ensure_option=True,
    disabled=False)

start_date_widget =widgets.DatePicker(
    description='Start date',
    disabled=False)

end_date_widget =widgets.DatePicker(
    description='End date',
    disabled=False)

input_file_widget = widgets.Text(description='File',placeholder='input.csv')

run_widget = widgets.Button(
    value=False,
    description='Click me',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check')

run_widget_mangos = widgets.Button(
    value=False,
    description=emoji.emojize('predict :mango:'),
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check')

gotosave_widget = widgets.Button(
    value=False,
    description='Go to save',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check')

gotomake_widget = widgets.Button(
    value=False,
    description='Go to make',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check')

gotoupload_widget = widgets.Button(
    value=False,
    description='Go to upload',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check')

gotoall_widget = widgets.Button(
    value=False,
    description='Go to all together',
    layout=Layout(width='20%', height='30px'),
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    style= {'button_color':'lightsteelblue'},
    tooltip='Description  ',
    icon='check')

manual_prediction_widget = widgets.Button(
    value=False,
    description='Predict',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check')


short_term_option_widget = widgets.Combobox(
    # value='John',
    placeholder='Choose option',
    options=['Follow-up','Special target with known start/end time',\
             'Special target (observe the most we can)','MANGOS','Dome rotation','Monitoring'],
    description='What kind? :',
    ensure_option=True,
    disabled=False)

short_day_widget =widgets.DatePicker(
    description='Day ',
    disabled=False)

short_start_time_widget = widgets.Text(description='Start time',placeholder='2020-01-16 23:00:00')
short_end_time_widget = widgets.Text(description='End time',placeholder='2020-01-17 03:00:00')

start_time_widget =widgets.DatePicker(
    description='Start Time',
    disabled=False)

end_time_widget =widgets.DatePicker(
    description='End Time',
    disabled=False)

#input_file_special_widget = widgets.Text(description='File',placeholder='input_short_term.csv')
 
input_target_list_widget = widgets.Text(description='File',
                                        placeholder=target_list_from_stargate_path)


schedules_st = SPOCKST.Schedules()
schedules_st.load_parameters()
schedules_mangos = SPOCKST.Schedules()
schedules_mangos.load_parameters(mangos=True)

    
target_widget = widgets.Combobox(
    # value='John',
    placeholder='Spxxx-xxxx',
    options=list(pd.read_csv(target_list_from_stargate_path,delimiter=',')['Sp_ID']), 
    description='Target name:',
    ensure_option=True,
    disabled=False)

target_special_widget = widgets.Combobox(
    # value='John',
    placeholder='Trappist-1x',
    options=list(schedules_st.target_table_spc['Sp_ID']), 
    description='Target special:',
    ensure_option=True,
    disabled=False)

target_follow_up_widget = widgets.Combobox(
    # value='John',
    placeholder='Trappist-1x',
    options=list(schedules_st.target_table_spc_follow_up['Sp_ID']), 
    description='Target follow-up:',
    ensure_option=True,
    disabled=False)

target_follow_up_mangos = widgets.Combobox(
    # value='John',
    placeholder='TOI-xxx',
    options=list(schedules_mangos.target_table_spc_follow_up['Sp_ID']), 
    description='MANGOs follow-up:',
    ensure_option=True,
    disabled=False)

plot_option_widget = widgets.Combobox(
    # value='John',
    placeholder='Choose a plot option',
    options=['Gant chart','Visibility plot SPECULOOS','Visibility plot follow-up',
             'Visibility plot special', 'Phase coverage','Global gant chart'],
    description='What kind? :',
    ensure_option=True,
    disabled=False)

#check_a_target_widget = widgets.Button(value=False,description='Select a target',disabled=False,
#        button_style='info',tooltip='show target list',icon='fa-arrow-right')

check_np_widget = widgets.Button(
    value=False,
    description='See night plans',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to see night plans',
    icon='check')

info_target_widget = widgets.Button(
    value=False,
    description='Info on a target',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to chose target',
    icon='check')


def run_all(b): 
    display(Javascript('Jupyter.notebook.execute_cells([3])'))
    
def run_button_mangos(b): 
    display(Javascript('Jupyter.notebook.execute_cells([3])'))
    
def run_gotosave(b): 
    display(Javascript('Jupyter.notebook.execute_cells([7])'))
def run_gotomake(b): 
    display(Javascript('Jupyter.notebook.execute_cells([9])'))
def run_gotoupload(b): 
    display(Javascript('Jupyter.notebook.execute_cells([11])'))
    
def run_manual_predict(b):
    display(Javascript('Jupyter.notebook.execute_cells([21])'))

# ----------------------------------------------------------- #
    
long_term_tab = widgets.Accordion(children=[widgets.HBox([observatory_widget,telescope_widget,operator_widget]),\
                                        widgets.HBox([start_date_widget,end_date_widget]), run_widget] )
long_term_tab.set_title(0, 'Observatory')
long_term_tab.set_title(1, 'Date range')
long_term_tab.set_title(2, 'Run long term')

# ----------------------------------------------------------- #

short_term_tab = widgets.Accordion(children=[widgets.HBox([observatory_widget,telescope_widget,operator_widget]),\
                                        widgets.VBox([widgets.HBox([short_term_option_widget,short_day_widget]),\
                                        widgets.HBox([short_start_time_widget,short_end_time_widget]),
                                             widgets.HBox([target_special_widget,target_follow_up_widget,
                                                          target_follow_up_mangos])]),
                                             run_widget])
short_term_tab.set_title(0, 'Observatory')
short_term_tab.set_title(1, 'Options')
short_term_tab.set_title(2, 'Run short term')

# ----------------------------------------------------------- #

plot_tab = widgets.Accordion(children=[plot_option_widget,run_widget])
plot_tab.set_title(0, 'Plot option')
plot_tab.set_title(1, 'Run plots')

# ----------------------------------------------------------- #

make_upload_tab = widgets.Accordion(children=[widgets.HBox([observatory_widget,telescope_widget]),\
                                          widgets.HBox([start_date_widget,end_date_widget]),\
                                          widgets.HBox([gotosave_widget,gotomake_widget,gotoupload_widget,\
                                                       gotoall_widget])])
make_upload_tab.set_title(0, 'Site info')
make_upload_tab.set_title(1, 'Date info')
make_upload_tab.set_title(2, 'Go to plan maker and upload')

# ----------------------------------------------------------- #

check_np_tab = widgets.Accordion(children=[check_np_widget])
make_upload_tab.set_title(0, 'Click')

# ----------------------------------------------------------- #

info_target_tab = widgets.Accordion(children=[info_target_widget])
make_upload_tab.set_title(0, 'Click')

# ----------------------------------------------------------- #

ra_text= widgets.FloatText(
    value=346.620833,
    description='RA:',
    disabled=False
)
dec_text =  widgets.FloatText(
    value=-5.0411111,
    description='DEC:',
    disabled=False
)
timing_text =  widgets.FloatText(
    value=2457322.51505,
    description='Timing (BJD):',
    disabled=False
)
period_text =  widgets.FloatText(
    value=1.51088212,
    description='Period (days):',
    disabled=False
)
duration_text =  widgets.FloatText(
    value=0.02511108,
    description='Duration (days):',
    disabled=False
)
name_widget = widgets.Text(description='Target name',placeholder='TOI-XXX')
prediction_date_widget =widgets.DatePicker(
    description='Start date',
    disabled=False)

nb_transits_widget = widgets.IntText(
    value=4,
    description='Nb transits:',
    disabled=False
)

coordinates = TwoByTwoLayout(top_left=ra_text, top_right=dec_text,
                     bottom_left=prediction_date_widget, bottom_right=name_widget)

ephemerids = TwoByTwoLayout(top_left=timing_text, top_right=period_text ,
                     bottom_left=duration_text, bottom_right=nb_transits_widget)

predict_tab = widgets.Accordion(children=[coordinates,ephemerids,manual_prediction_widget,
                                          run_widget])

predict_tab.set_title(0, 'Coordinates')
predict_tab.set_title(1, 'Ephemerids')
predict_tab.set_title(2, 'Run predictions')
predict_tab.set_title(3, 'From WG6 directly')

# ----------------------------------------------------------- #

mango_tab = widgets.Accordion(children=[run_widget_mangos])
mango_tab.set_title(5, 'Predictions MANGOS')


# ----------------------------------------------------------- #
run_widget.on_click(run_all)
run_widget_mangos.on_click(run_button_mangos)
gotosave_widget.on_click(run_gotosave)
gotomake_widget.on_click(run_gotomake)
gotoupload_widget.on_click(run_gotoupload)
gotoall_widget.on_click(run_all)
check_np_widget.on_click(run_all)
info_target_widget.on_click(run_all)

manual_prediction_widget.on_click(run_manual_predict)

tab_nest = widgets.Tab()
tab_nest.children = [long_term_tab, short_term_tab,plot_tab,make_upload_tab,check_np_tab,info_target_tab, predict_tab, mango_tab]
tab_nest.set_title(0, 'Long term scheduler ')
tab_nest.set_title(1, 'Short term scheduler')
tab_nest.set_title(2, 'Plots')
tab_nest.set_title(3, 'Make & Upload only')
tab_nest.set_title(4, 'See night plans')
tab_nest.set_title(5, 'Info on a target')
tab_nest.set_title(6, 'Predictions')
tab_nest.set_title(7, emoji.emojize('Predictions :mango:'))
tab_nest

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [104]:
# CELL 3

if tab_nest.selected_index == 0:
    if observatory_widget.value is '':
        sys.exit('ERROR: You must choose a site')
    if start_date_widget.value is '' and end_date_widget.value is '':
        sys.exit('ERROR: The start and/or end date are not valid')

    start_date = Time(datetime.datetime.strptime(str(start_date_widget.value), '%Y-%m-%d')+datetime.timedelta(hours=15)).iso
    end_date = Time(datetime.datetime.strptime(str(end_date_widget.value), '%Y-%m-%d')+datetime.timedelta(hours=15)).iso
    schedule = SPOCKLT.Schedules()
    schedule.observatory_name = observatory_widget.value
    schedule.load_parameters(date_range=[start_date,end_date])
    
    if schedule.observatory_name == 'SSO':
        schedule.telescope = str(telescope_widget.value)
    else:
        schedule.telescope = schedule.observatory_name
    if schedule.observatory_name == 'Saint-Ex':
        schedule.telescope = 'Saint-Ex'
        schedule.make_schedule(Altitude_constraint = 28, Moon_constraint = 30)
        display(schedule.target_table_spc[schedule.idx_first_target])
        display(schedule.target_table_spc[schedule.idx_second_target])
        display(Javascript('Jupyter.notebook.execute_cells([13])'))
    else:
        if schedule.observatory_name == 'SNO':
            schedule.telescope = 'Artemis'
        schedule.make_schedule(Altitude_constraint = 25, Moon_constraint = 30)
        display(schedule.target_table_spc[schedule.idx_first_target])
        display(schedule.target_table_spc[schedule.idx_second_target])
        display(Javascript('Jupyter.notebook.execute_cells([13])'))

if tab_nest.selected_index == 1:
    if observatory_widget.value is '':
        sys.exit('ERROR: You must choose a site')
    if short_term_option_widget.value == 'MANGOS':
        schedule = SPOCKST.Schedules()
        schedule.load_parameters(mangos=True)
    else:
        schedule = SPOCKST.Schedules()
        schedule.load_parameters()
    
    if observatory_widget.value == 'SSO':
        schedule.altitude_constraint = 25
        schedule.telescope = str(telescope_widget.value)
    if observatory_widget.value == 'SNO':
        schedule.altitude_constraint = 25
        schedule.telescope = 'Artemis'
    if observatory_widget.value == 'Saint-Ex':
        schedule.altitude_constraint = 28
        schedule.telescope = 'Saint-Ex'
        
    schedule.use = short_term_option_widget.value
    day = Time(datetime.datetime.combine(short_day_widget.value,datetime.time(15,0,0)),format='datetime').iso
    schedule.day_of_night = Time(day)
    
    if short_term_option_widget.value == 'Follow-up':
        if target_follow_up_widget.value != '':
            schedule.observatory_name = observatory_widget.value
            schedule.transit_follow_up(input_name=target_follow_up_widget.value)
        else:
            schedule.observatory_name = observatory_widget.value
            schedule.transit_follow_up()
            
    if short_term_option_widget.value == 'MANGOS':
        if target_follow_up_mangos.value != '':
            schedule.observatory_name = observatory_widget.value
            schedule.transit_follow_up(input_name=target_follow_up_mangos.value)
        else:
            schedule.observatory_name = observatory_widget.value
            schedule.transit_follow_up()

            
    if short_term_option_widget.value == 'Special target with known start/end time':
        start_date = [Time(short_start_time_widget.value).iso]
        end_date = [Time(short_end_time_widget.value).iso]
        schedule.start_end_range = Time([start_date,end_date])
        schedule.observatory_name = observatory_widget.value
        input_name = target_special_widget.value
        schedule.special_target_with_start_end(input_name=input_name)
        
    if short_term_option_widget.value == 'Special target (observe the most we can)':
        input_name = target_special_widget.value
        schedule.observatory_name = observatory_widget.value
        schedule.special_target(input_name=input_name)
        
    if short_term_option_widget.value == 'Dome rotation':
        schedule.dome_rotation()
        
    if short_term_option_widget.value == 'Monitoring':
        start_date = Time(short_start_time_widget.value).iso
        end_date = Time(short_end_time_widget.value).iso
        schedule.start_end_range = Time([start_date,end_date])
        input_name = target_special_widget.value
        schedule.monitoring(input_name,5,61)
        
    display(schedule.SS1_night_blocks)
    
    try:
        schedule.make_scheduled_table()
        schedule.planification()
        schedule.make_night_block()
        display(schedule.scheduled_table_sorted)
        display(Javascript("Jupyter.notebook.execute_cells([13])"))
    except (SystemExit) as e :
        display(Javascript("Jupyter.notebook.execute_cells([4])"))
        print(Fore.YELLOW + 'WARNING: ' + Fore.BLACK + 'No existing night_block for this night or target already scheduled that night')

    #display(Javascript('Jupyter.notebook.execute_cells([6])'))
    
if tab_nest.selected_index == 2:   
    if plot_option_widget.value == 'Visibility plot SPECULOOS':
        display(Javascript('Jupyter.notebook.execute_cells([5])'))
        
    if plot_option_widget.value == 'Visibility plot follow-up':
        display(Javascript('Jupyter.notebook.execute_cells([22])'))
                
    if plot_option_widget.value == 'Visibility plot special':
        display(Javascript('Jupyter.notebook.execute_cells([23])'))
        
    if plot_option_widget.value == 'Phase coverage':
        display(Javascript('Jupyter.notebook.execute_cells([17])'))

    if plot_option_widget.value == 'Global gant chart':
        schedule = SPOCKLT.Schedules()
        schedule.observatory_name = 'SSO'
        schedule.load_parameters(date_range=[start_date,end_date])
        SPOCKplot.gantt_chart_all(schedule.target_list)
        
    if plot_option_widget.value == 'Gant chart':
        display(Javascript("Jupyter.notebook.execute_cells([6])"))

if tab_nest.selected_index == 3:
    if observatory_widget.value is '':
        sys.exit('ERROR: You must choose a site')
    if start_date_widget.value is '' and end_date_widget.value is '':
        sys.exit('ERROR: The start and/or end date are not valid')
    else:
        start_date = Time(datetime.datetime.strptime(str(start_date_widget.value), '%Y-%m-%d')+datetime.timedelta(hours=15)).iso
        end_date = Time(datetime.datetime.strptime(str(end_date_widget.value), '%Y-%m-%d')+datetime.timedelta(hours=15)).iso
        schedule = SPOCKLT.Schedules()
        schedule.observatory_name = observatory_widget.value
        schedule.load_parameters(date_range=[start_date,end_date])
        schedule.date_range = Time([start_date,end_date])
        if observatory_widget.value == 'SSO':
            schedule.telescope = str(telescope_widget.value)
    display(Javascript("Jupyter.notebook.execute_cells([13])"))

if tab_nest.selected_index == 4:
    display(Javascript('Jupyter.notebook.execute_cells([18])'))
    
if tab_nest.selected_index == 5:
    display(Javascript('Jupyter.notebook.execute_cells([19])'))
    
if tab_nest.selected_index == 6:
    display(Javascript('Jupyter.notebook.execute_cells([20])'))
    
if tab_nest.selected_index == 7:
    schedules_st = SPOCKST.Schedules()
    schedules_st.load_parameters(mangos=True)
    display(Javascript('Jupyter.notebook.execute_cells([24])'))


Updating hours of obs :   0%|          | 0/31 [00:00<?, ?it/s]

Scheduling :   0%|          | 0/31 [00:00<?, ?it/s]

INFO:  day is :  2023-05-01 15:00:00.000
INFO:  Not using moon phase in ETC
INFO:  Not using moon phase in ETC
INFO:  First target is 'rise'


INFO:  day is :  2023-05-02 15:00:00.000
INFO:  Not using moon phase in ETC
INFO:  Not using moon phase in ETC
INFO:  First target is 'rise'


INFO:  Local path does not exist yet 
INFO:  situation 9
INFO:  situation 4
INFO:  situation 2
INFO:  no transition block
INFO:  day is :  2023-05-03 15:00:00.000
INFO:  Not using moon phase in ETC
INFO:  Not using moon phase in ETC
INFO:  First target is 'rise'


INFO:  day is :  2023-05-04 15:00:00.000
INFO:  Not using moon phase in ETC
INFO:  Not using moon phase in ETC
INFO:  First target is 'rise'


INFO:  day is :  2023-05-05 15:00:00.000
INFO:  Not using moon phase in ETC
INFO:  Not using moon phase in ETC
INFO:  First target is 'rise'


INFO:  day is :  2023-05-06 15:00:00.000
INFO:  Not using moon phase in ETC
INFO:  Not using moon phase in ETC
INFO:  First target is 'rise'


INFO:  day is :  202

INFO:  Not using moon phase in ETC
INFO:  First target is 'set'


INFO:  day is :  2023-05-30 15:00:00.000
INFO:  Not using moon phase in ETC
INFO:  Not using moon phase in ETC
INFO:  First target is 'set'


INFO:  day is :  2023-05-31 15:00:00.000
INFO:  Not using moon phase in ETC
INFO:  Not using moon phase in ETC
INFO:  First target is 'set'




id,Gaia_ID,soi,program,Sp_ID,RA,DEC,num_stel,num_plan,num_spec,num_img,fc_url,last_mod,last_mod_by,mag_g,J,mag_h,mag_ks,mag_g_err,mag_j_err,mag_h_err,mag_ks_err,nb_hours_surved,Program,SNR_JWST_HZ_tr,SpT,spt_err,cf,teff,teff_error,dist,dist_error,mass,mass_error,radius,radius_error,lum,lum_error,phot_spt,phot_spt_ref,spec_spt,spec_spt_ref,Unnamed: 42,texp_spc,Filter_spc,texp_trap,Filter_trap,telescope,nb_hours_threshold,SNR_TESS_temp,SNR_Spec_temp,SNR_SPIRIT,texp_spirit
int64,int64,int64,int64,str13,float64,float64,int64,int64,int64,int64,float64,str19,str10,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str9,str19,str15,str22,float64,int64,str3,int64,str3,float64,int64,int64,int64,float64,float64
1125,5884382654716110848,667,1,Sp1546-5534,236.672867,-55.579857,0,0,0,0,--,2021-03-19 21:26:27,dsebastian,14.576,10.21,9.55,9.11,0.001,0.022,0.022,0.019,166.9886724844583,1,6.32,6.5,1.0,0.44,2758.0,105.0,8.4,0.0,0.1,0.001,0.121,0.01,0.0007643233090566,2.346728633209604e-05,--,--,M7.5,2013A&A...556A..15R,--,20,i,0,I+z,--,200,0,0,12.044955401107757,2.82649504249974


id,Gaia_ID,soi,program,Sp_ID,RA,DEC,num_stel,num_plan,num_spec,num_img,fc_url,last_mod,last_mod_by,mag_g,J,mag_h,mag_ks,mag_g_err,mag_j_err,mag_h_err,mag_ks_err,nb_hours_surved,Program,SNR_JWST_HZ_tr,SpT,spt_err,cf,teff,teff_error,dist,dist_error,mass,mass_error,radius,radius_error,lum,lum_error,phot_spt,phot_spt_ref,spec_spt,spec_spt_ref,Unnamed: 42,texp_spc,Filter_spc,texp_trap,Filter_trap,telescope,nb_hours_threshold,SNR_TESS_temp,SNR_Spec_temp,SNR_SPIRIT,texp_spirit
int64,int64,int64,int64,str13,float64,float64,int64,int64,int64,int64,float64,str19,str10,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str9,str19,str15,str22,float64,int64,str3,int64,str3,float64,int64,int64,int64,float64,float64
1399,6421389047155380352,815,1,Sp1855-6914,283.953064,-69.23748,0,0,0,0,--,2021-03-19 21:26:27,dsebastian,14.253,10.47,9.88,9.51,0.0,0.021,0.021,0.021,27.626692365404196,1,4.83,5.5,1.0,0.09,2900.0,104.0,11.6,0.0,0.11,0.001,0.136,0.02,0.0011790638480778,4.127088246453611e-05,--,--,M6e,2017A&A...600A..19P,--,25,i,0,I+z,--,200,0,0,8.94703307034287,3.856361019078496


<IPython.core.display.Javascript object>

In [ ]:
# CELL 4 -- Lock block for future obs
lock_target_widget = widgets.Button(
    value=False,
    description='Lock block',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Save',
    icon='check'
)
def run_lock(b): 
    schedule.locking_observations()

display(lock_target_widget)

lock_target_widget.on_click(run_lock)


In [ ]:
# CELL 5 -- Plot visibility plot
@interact
def plt_target(site=['Saint-Ex','SSO','SNO','TS_La_Silla','TN_Oukaimeden'],\
               Target=list(pd.read_csv(target_list_from_stargate_path,delimiter=',')['Sp_ID']),\
               day_widget = widgets.DatePicker(description='Pick a Date')):
    try:
        day = Time(datetime.datetime.combine(day_widget,datetime.time(15,0,0)),format='datetime')
        SPOCKplot.airmass_altitude_plot_given_target(name_observatory=site,\
                                                     day=day,\
                                                     target=Target,\
                                                     path_target_list=target_list_from_stargate_path)

    except TypeError:
        print('ERROR: Choose a date ! ')

In [ ]:
# CELL 6 -- Plot gant chart
@interact
def plt_gant_chart(Telescope=['Europa','Io','Callisto','Ganymede','Artemis','Saint-Ex','TS_La_Silla','TN_Oukaimeden','all_speculoos'],start_widget = widgets.DatePicker(description='Start Date'),end_widget = widgets.DatePicker(description='End Date')):
    try:
        start_day = Time(datetime.datetime.combine(start_widget,datetime.time(15,0,0)),format='datetime')
        end_day = Time(datetime.datetime.combine(end_widget,datetime.time(15,0,0)),format='datetime')
        if Telescope == 'all_speculoos':
            Telescope = ['Europa','Io','Callisto','Ganymede','Artemis'] # 'Saint-Ex'
        SPOCKplot.gantt_chart(start_day,end_day,Telescope)
    except (TypeError,NameError) as e:
        print('ERROR: Choose date range ! '),

In [ ]:
# CELL 7 -- BUTTON -- Save plans
save_widget = widgets.Button(
    value=False,
    description='Save plans',
    layout=Layout(width='20%', height='40px'),
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    style= {'button_color':'palegoldenrod'},
    tooltip='Save',
    icon='check'
)
def run_save(b): 
    display(Javascript('Jupyter.notebook.execute_cells([8])'))

display(save_widget)

save_widget.on_click(run_save)


In [ ]:
# CELL 8 -- Save Plans
start_date = Time(datetime.datetime.strptime(str(start_date_widget.value), '%Y-%m-%d')+datetime.timedelta(hours=15)).iso
end_date = Time(datetime.datetime.strptime(str(end_date_widget.value), '%Y-%m-%d')+datetime.timedelta(hours=15)).iso
schedule = SPOCKLT.Schedules()
schedule.observatory_name = observatory_widget.value
schedule.load_parameters(date_range=[start_date,end_date])
schedule.date_range = Time([start_date,end_date])

if tab_nest.selected_index == 0:
    schedule.telescope = telescope_widget.value
    if schedule.observatory.name == 'SNO':
        schedule.telescope = 'Artemis'
        SPOCKLT.save_schedule(save=True, over_write=True, date_range=schedule.date_range, 
                              telescope=schedule.telescope)
        
    if schedule.observatory.name == 'Saint-Ex':
        schedule.telescope = 'Saint-Ex'
        SPOCKLT.save_schedule(save=True, over_write=True, date_range=schedule.date_range, 
                              telescope=schedule.telescope)
        
    if schedule.observatory.name == 'SSO':
        SPOCKLT.save_schedule(save=True, over_write=True, date_range=schedule.date_range, 
                              telescope=schedule.telescope)

    if schedule.observatory.name == 'TS_La_Silla':
        schedule.telescope = 'TS_La_Silla'
        SPOCKLT.save_schedule(save=True, over_write=True, date_range=schedule.date_range, 
                              telescope=schedule.telescope)
        
    if schedule.observatory.name == 'TN_Oukaimeden':
        schedule.telescope = 'TN_Oukaimeden'
        SPOCKLT.save_schedule(save=True, over_write=True, date_range=schedule.date_range, 
                              telescope=schedule.telescope)

if tab_nest.selected_index == 1:
    schedule.telescope = telescope_widget.value
    if schedule.observatory.name == 'SNO':
        schedule.telescope = 'Artemis'
        SPOCKST.save_schedule(save=True, over_write=True, day=schedule.day_of_night, 
                              telescope=schedule.telescope)

    if schedule.observatory.name == 'Saint-Ex':
        schedule.telescope = 'Saint-Ex'
        SPOCKST.save_schedule(save=True, over_write=True, day=schedule.day_of_night, 
                              telescope=schedule.telescope)
          
    if schedule.observatory.name == 'SSO':
        SPOCKST.save_schedule(save=True, over_write=True, day=schedule.day_of_night, 
                              telescope=schedule.telescope)
         
    if schedule.observatory.name == 'TS_La_Silla':
        schedule.telescope = 'TS_La_Silla'
        SPOCKST.save_schedule(save=True, over_write=True, day=schedule.day_of_night, 
                              telescope=schedule.telescope)
        
    if schedule.observatory.name == 'TN_Oukaimeden':
        schedule.telescope = 'TN_Oukaimeden'
        SPOCKST.save_schedule(save=True, over_write=True, day=schedule.day_of_night, 
                              telescope=schedule.telescope)
    

In [ ]:
# CELL 9 -- BUTTON -- Make plans
makeplans_widget = widgets.Button(
    value=False,
    description='Make ACP plans',
    layout=Layout(width='20%', height='40px'),
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    style= {'button_color':'palegreen'},
    tooltip='Make',
    icon='check'
)
def run_make(b): 
    display(Javascript('Jupyter.notebook.execute_cells([10])'))

display(makeplans_widget)

makeplans_widget.on_click(run_make)


In [ ]:
# CELL 10 -- Make plans
start_date = Time(datetime.datetime.strptime(str(start_date_widget.value), '%Y-%m-%d')+datetime.timedelta(hours=15)).iso
end_date = Time(datetime.datetime.strptime(str(end_date_widget.value), '%Y-%m-%d')+datetime.timedelta(hours=15)).iso
schedule = SPOCKLT.Schedules()
schedule.observatory_name = observatory_widget.value
schedule.load_parameters(date_range=[start_date,end_date])
schedule.date_range = Time([start_date,end_date])

if tab_nest.selected_index == 0:
    schedule.telescope = telescope_widget.value
    if schedule.observatory.name == 'SNO':
        schedule.telescope = 'Artemis'
        SPOCKLT.make_plans(day=schedule.date_range[0], nb_days=schedule.date_range_in_days, 
                           telescope=schedule.telescope)

    if schedule.observatory.name == 'Saint-Ex':
        schedule.telescope = 'Saint-Ex'
        SPOCKLT.make_plans(day=schedule.date_range[0], nb_days=schedule.date_range_in_days, 
                           telescope=schedule.telescope)

    if schedule.observatory.name == 'SSO':
        SPOCKLT.make_plans(day=schedule.date_range[0], nb_days=schedule.date_range_in_days, 
                           telescope=schedule.telescope)
    display(Javascript('Jupyter.notebook.execute_cells([11])'))

if tab_nest.selected_index == 1:
    schedule.telescope = telescope_widget.value
    if schedule.observatory.name == 'SNO':
        schedule.telescope = 'Artemis'
        SPOCKST.make_plans(day=schedule.day_of_night, nb_days=1, 
                           telescope=schedule.telescope)
        
    if schedule.observatory.name == 'Saint-Ex':
        schedule.telescope = 'Saint-Ex'
        SPOCKST.make_plans(day=schedule.day_of_night, nb_days=1, 
                           telescope=schedule.telescope)
        
    if schedule.observatory.name == 'SSO':
        SPOCKST.make_plans(day=schedule.day_of_night, nb_days=1, 
                           telescope=schedule.telescope)
    display(Javascript('Jupyter.notebook.execute_cells([11])'))
        
if tab_nest.selected_index == 3:
    schedule.telescope = telescope_widget.value
    if schedule.observatory.name == 'SNO':
        schedule.telescope = 'Artemis'
        SPOCKLT.make_plans(day=schedule.date_range[0], nb_days=schedule.date_range_in_days, 
                           telescope=schedule.telescope)
        
    if schedule.observatory.name == 'Saint-Ex':
        schedule.telescope = 'Saint-Ex'
        SPOCKLT.make_plans(day=schedule.date_range[0], nb_days=schedule.date_range_in_days, 
                           telescope=schedule.telescope)

    if schedule.observatory.name == 'SSO':
        SPOCKLT.make_plans(day=schedule.date_range[0], nb_days=schedule.date_range_in_days, 
                           telescope=schedule.telescope)
        
    if schedule.observatory.name == 'TS_La_Silla':
        schedule.telescope = 'TS_La_Silla'
        SPOCKLT.make_plans(day=schedule.date_range[0], nb_days=schedule.date_range_in_days, 
                           telescope=schedule.telescope)
        
    if schedule.observatory.name == 'TN_Oukaimeden':
        schedule.telescope = 'TN_Oukaimeden'
        SPOCKLT.make_plans(day=schedule.date_range[0], nb_days=schedule.date_range_in_days, 
                           telescope=schedule.telescope)
    display(Javascript('Jupyter.notebook.execute_cells([11])'))

In [ ]:
# CELL 11 -- BUTTON -- Upload plans
uploadplans_widget = widgets.Button(
    value=False,
    description='Upload ACP plans/night-blocks',
    layout=Layout(width='30%', height='40px'),
    
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    style= {'button_color':'lightblue'},
    tooltip='Upload',
    icon='check'
)
def run_upload(b): 
    display(Javascript('Jupyter.notebook.execute_cells([12])'))

display(uploadplans_widget)

uploadplans_widget.on_click(run_upload)

In [ ]:
# CELL 12 -- Upload plans
start_date = Time(datetime.datetime.strptime(str(start_date_widget.value), '%Y-%m-%d')+datetime.timedelta(hours=15)).iso
end_date = Time(datetime.datetime.strptime(str(end_date_widget.value), '%Y-%m-%d')+datetime.timedelta(hours=15)).iso
schedule = SPOCKLT.Schedules()
schedule.observatory_name = observatory_widget.value
schedule.load_parameters(date_range=[start_date,end_date])
schedule.date_range = Time([start_date,end_date])

if tab_nest.selected_index == 0:
    schedule.telescope = telescope_widget.value
    if schedule.observatory.name == 'SNO':
        schedule.telescope = 'Artemis'
        SPOCKLT.upload_plans(day=schedule.date_range[0], nb_days=schedule.date_range_in_days,
                             telescope = schedule.telescope)

    if schedule.observatory.name == 'Saint-Ex':
        schedule.telescope = 'Saint-Ex'
        SPOCKLT.upload_plans(day=schedule.date_range[0], nb_days=schedule.date_range_in_days,
                             telescope = schedule.telescope)
        
    if schedule.observatory.name == 'SSO':
        SPOCKLT.upload_plans(day=schedule.date_range[0], nb_days=schedule.date_range_in_days,
                             telescope = schedule.telescope)
if tab_nest.selected_index == 1:
    schedule.telescope = telescope_widget.value
    if schedule.observatory.name == 'SNO':
        schedule.telescope = 'Artemis'
        SPOCKST.upload_plans(day=schedule.day_of_night, nb_days=1,
                             telescope = schedule.telescope)
        
    if schedule.observatory.name == 'Saint-Ex':
        schedule.telescope = 'Saint-Ex'
        SPOCKST.upload_plans(day=schedule.day_of_night, nb_days=1,
                             telescope = schedule.telescope)
       
        
    if schedule.observatory.name == 'SSO':
        SPOCKST.upload_plans(day=schedule.day_of_night, nb_days=1,
                             telescope = schedule.telescope)
        
if tab_nest.selected_index == 3:
    schedule.telescope = telescope_widget.value
    if schedule.observatory.name == 'SNO':
        schedule.telescope = 'Artemis'
        SPOCKLT.upload_plans(day=schedule.date_range[0], nb_days=schedule.date_range_in_days,
                             telescope = schedule.telescope)

    if schedule.observatory.name == 'Saint-Ex':
        schedule.telescope = 'Saint-Ex'
        SPOCKLT.upload_plans(day=schedule.date_range[0], nb_days=schedule.date_range_in_days,
                             telescope = schedule.telescope)
        
    if schedule.observatory.name == 'SSO':
        SPOCKLT.upload_plans(day=schedule.date_range[0], nb_days=schedule.date_range_in_days,
                             telescope = schedule.telescope)

    if schedule.observatory.name == 'TS_La_Silla':
        schedule.telescope = 'TS_La_Silla'
        SPOCKLT.upload_plans(day=schedule.date_range[0], nb_days=schedule.date_range_in_days,
                             telescope = schedule.telescope)
        
    if schedule.observatory.name == 'TN_Oukaimeden':
        schedule.telescope = 'TN_Oukaimeden'
        SPOCKLT.upload_plans(day=schedule.date_range[0], nb_days=schedule.date_range_in_days,
                             telescope = schedule.telescope)


In [105]:
# CELL 13 -- BUTTON -- Save plans, Make plans, Upload plans
smu_plans_widget = widgets.Button(
    value=False,
    description='Save, make, upload',
    layout=Layout(width='20%', height='40px'),
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='',
    icon='check'
)
def run_savemakeupload(b): 
    display(Javascript('Jupyter.notebook.execute_cells([14])'))

display(smu_plans_widget)

smu_plans_widget.on_click(run_savemakeupload)

Button(description='Save, make, upload', icon='check', layout=Layout(height='40px', width='20%'), style=Button…

<IPython.core.display.Javascript object>

In [106]:
# CELL 14 -- Save Plans, Make plans, and Upload plans
if tab_nest.selected_index == 0:
    schedule.telescope = telescope_widget.value
    if schedule.observatory.name == 'SNO':
        schedule.telescope = 'Artemis'
        SPOCKLT.save_schedule(save=True, over_write=True, date_range=schedule.date_range, 
                              telescope=schedule.telescope)
        SPOCKLT.make_plans(day=schedule.date_range[0], nb_days=schedule.date_range_in_days, 
                           telescope=schedule.telescope)
        SPOCKLT.upload_plans(day=schedule.date_range[0], nb_days=schedule.date_range_in_days,
                             telescope = schedule.telescope)

    if schedule.observatory.name == 'Saint-Ex':
        schedule.telescope = 'Saint-Ex'
        SPOCKLT.save_schedule(save=True, over_write=True, date_range=schedule.date_range, 
                              telescope=schedule.telescope)
        SPOCKLT.make_plans(day=schedule.date_range[0], nb_days=schedule.date_range_in_days, 
                           telescope=schedule.telescope)
        SPOCKLT.upload_plans(day=schedule.date_range[0], nb_days=schedule.date_range_in_days,
                             telescope = schedule.telescope)
        
    if schedule.observatory.name == 'SSO':
        SPOCKLT.save_schedule(save=True, over_write=True, date_range=schedule.date_range, 
                              telescope=schedule.telescope)
        SPOCKLT.make_plans(day=schedule.date_range[0], nb_days=schedule.date_range_in_days, 
                           telescope=schedule.telescope)
        SPOCKLT.upload_plans(day=schedule.date_range[0], nb_days=schedule.date_range_in_days,
                             telescope = schedule.telescope)

    if schedule.observatory.name == 'TS_La_Silla':
        schedule.telescope = 'TS_La_Silla'
        SPOCKLT.save_schedule(save=True, over_write=True, date_range=schedule.date_range, 
                              telescope=schedule.telescope)
        display(Javascript("Jupyter.notebook.execute_cells([9])"))
        
    if schedule.observatory.name == 'TN_Oukaimeden':
        schedule.telescope = 'TN_Oukaimeden'
        SPOCKLT.save_schedule(save=True, over_write=True, date_range=schedule.date_range, 
                              telescope=schedule.telescope)
        display(Javascript("Jupyter.notebook.execute_cells([9])"))

if tab_nest.selected_index == 1:
    schedule.telescope = telescope_widget.value
    if schedule.observatory.name == 'SNO':
        schedule.telescope = 'Artemis'
        SPOCKST.save_schedule(save=True, over_write=True, day=schedule.day_of_night, 
                              telescope=schedule.telescope)
        SPOCKST.make_plans(day=schedule.day_of_night, nb_days=1, 
                           telescope=schedule.telescope)
        SPOCKST.upload_plans(day=schedule.day_of_night, nb_days=1,
                             telescope = schedule.telescope)
        
    if schedule.observatory.name == 'Saint-Ex':
        schedule.telescope = 'Saint-Ex'
        SPOCKST.save_schedule(save=True, over_write=True, day=schedule.day_of_night, 
                              telescope=schedule.telescope)
        SPOCKST.make_plans(day=schedule.day_of_night, nb_days=1, 
                           telescope=schedule.telescope)
        SPOCKST.upload_plans(day=schedule.day_of_night, nb_days=1,
                             telescope = schedule.telescope)
       
        
    if schedule.observatory.name == 'SSO':
        SPOCKST.save_schedule(save=True, over_write=True, day=schedule.day_of_night, 
                              telescope=schedule.telescope)
        SPOCKST.make_plans(day=schedule.day_of_night, nb_days=1, 
                           telescope=schedule.telescope)
        SPOCKST.upload_plans(day=schedule.day_of_night, nb_days=1,
                             telescope = schedule.telescope)
         
    if schedule.observatory.name == 'TS_La_Silla':
        schedule.telescope = 'TS_La_Silla'
        SPOCKST.save_schedule(save=True, over_write=True, day=schedule.day_of_night, 
                              telescope=schedule.telescope)
        display(Javascript("Jupyter.notebook.execute_cells([9])"))
        
    if schedule.observatory.name == 'TN_Oukaimeden':
        schedule.telescope = 'TN_Oukaimeden'
        SPOCKST.save_schedule(save=True, over_write=True, day=schedule.day_of_night, 
                              telescope=schedule.telescope)
        display(Javascript("Jupyter.notebook.execute_cells([9])"))
        
if tab_nest.selected_index == 3:
    schedule.telescope = telescope_widget.value
    if schedule.observatory.name == 'SNO':
        schedule.telescope = 'Artemis'
        SPOCKLT.make_plans(day=schedule.date_range[0], nb_days=schedule.date_range_in_days, 
                           telescope=schedule.telescope)
        SPOCKLT.upload_plans(day=schedule.date_range[0], nb_days=schedule.date_range_in_days,
                             telescope = schedule.telescope)

    if schedule.observatory.name == 'Saint-Ex':
        schedule.telescope = 'Saint-Ex'
        SPOCKLT.make_plans(day=schedule.date_range[0], nb_days=schedule.date_range_in_days, 
                           telescope=schedule.telescope)
        SPOCKLT.upload_plans(day=schedule.date_range[0], nb_days=schedule.date_range_in_days,
                             telescope = schedule.telescope)
        
    if schedule.observatory.name == 'SSO':
        SPOCKLT.make_plans(day=schedule.date_range[0], nb_days=schedule.date_range_in_days, 
                           telescope=schedule.telescope)
        SPOCKLT.upload_plans(day=schedule.date_range[0], nb_days=schedule.date_range_in_days,
                             telescope = schedule.telescope)

    if schedule.observatory.name == 'TS_La_Silla':
        schedule.telescope = 'TS_La_Silla'
        SPOCKLT.make_plans(day=schedule.date_range[0], nb_days=schedule.date_range_in_days, 
                           telescope=schedule.telescope)
        SPOCKLT.upload_plans(day=schedule.date_range[0], nb_days=schedule.date_range_in_days,
                             telescope = schedule.telescope)
        
    if schedule.observatory.name == 'TN_Oukaimeden':
        schedule.telescope = 'TN_Oukaimeden'
        SPOCKLT.make_plans(day=schedule.date_range[0], nb_days=schedule.date_range_in_days, 
                           telescope=schedule.telescope)
        SPOCKLT.upload_plans(day=schedule.date_range[0], nb_days=schedule.date_range_in_days,
                             telescope = schedule.telescope)


INFO:  "/Users/ed268546/Documents/codes/SPOCK/night_blocks_propositions/night_blocks_Io_2023-05-01.txt" has been copied to "/Users/ed268546/Documents/codes/SPOCK/DATABASE/Io/Archive_night_blocks/"
INFO:  "/Users/ed268546/Documents/codes/SPOCK/night_blocks_propositions/night_blocks_Io_2023-05-02.txt" has been copied to "/Users/ed268546/Documents/codes/SPOCK/DATABASE/Io/Archive_night_blocks/"
INFO:  "/Users/ed268546/Documents/codes/SPOCK/night_blocks_propositions/night_blocks_Io_2023-05-03.txt" has been copied to "/Users/ed268546/Documents/codes/SPOCK/DATABASE/Io/Archive_night_blocks/"
INFO:  "/Users/ed268546/Documents/codes/SPOCK/night_blocks_propositions/night_blocks_Io_2023-05-04.txt" has been copied to "/Users/ed268546/Documents/codes/SPOCK/DATABASE/Io/Archive_night_blocks/"
INFO:  "/Users/ed268546/Documents/codes/SPOCK/night_blocks_propositions/night_blocks_Io_2023-05-05.txt" has been copied to "/Users/ed268546/Documents/codes/SPOCK/DATABASE/Io/Archive_night_blocks/"
INFO:  "/Users/

INFO:  Path exists and is:  /Users/ed268546/Documents/codes/SPOCK/DATABASE/Io/night_blocks_Io_2023-05-15.txt

INFO:  Path exists and is:  /Users/ed268546/Documents/codes/SPOCK/DATABASE/Io/night_blocks_Io_2023-05-16.txt

INFO:  Path exists and is:  /Users/ed268546/Documents/codes/SPOCK/DATABASE/Io/night_blocks_Io_2023-05-17.txt

INFO:  Path exists and is:  /Users/ed268546/Documents/codes/SPOCK/DATABASE/Io/night_blocks_Io_2023-05-18.txt

INFO:  Path exists and is:  /Users/ed268546/Documents/codes/SPOCK/DATABASE/Io/night_blocks_Io_2023-05-19.txt

INFO:  Path exists and is:  /Users/ed268546/Documents/codes/SPOCK/DATABASE/Io/night_blocks_Io_2023-05-20.txt

INFO:  Path exists and is:  /Users/ed268546/Documents/codes/SPOCK/DATABASE/Io/night_blocks_Io_2023-05-21.txt

INFO:  Path exists and is:  /Users/ed268546/Documents/codes/SPOCK/DATABASE/Io/night_blocks_Io_2023-05-22.txt

INFO:  Path exists and is:  /Users/ed268546/Documents/codes/SPOCK/DATABASE/Io/night_blocks_Io_2023-05-23.txt

INFO:  Pat

kex_exchange_identification: read: Connection reset by peer
kex_exchange_identification: read: Connection reset by peer
lost connection
lost connection
kex_exchange_identification: Connection closed by remote host
lost connection
kex_exchange_identification: read: Connection reset by peer
lost connection
kex_exchange_identification: read: Connection reset by peer


-----> 2023-05-04 Zip Plans_by_dates folder uploaded on the HUB for Io
-----> 2023-05-05 Plans uploaded on the Cambridge server
-----> 2023-05-05 Night plans uploaded on the Cambridge server
-----> 2023-05-05 Zip Plans_by_dates folder uploaded on the Cambridge server
-----> 2023-05-05 Zip Plans_by_dates folder uploaded on the HUB for Io
-----> 2023-05-06 Plans uploaded on the Cambridge server
-----> 2023-05-06 Night plans uploaded on the Cambridge server
-----> 2023-05-06 Zip Plans_by_dates folder uploaded on the Cambridge server


lost connection
kex_exchange_identification: Connection closed by remote host
lost connection


...

Permission denied, please try again.
kex_exchange_identification: read: Connection reset by peer
lost connection
ssh: connect to host appcs.ra.phy.cam.ac.uk port 22: Connection refused
ssh: connect to host appcs.ra.phy.cam.ac.uk port 22: Connection refused
lost connection
ssh: connect to host appcs.ra.phy.cam.ac.uk port 22: Connection refused
lost connection
Permission denied, please try again.
Permission denied, please try again.


 Plans uploaded on the Cambridge server
-----> 2023-05-29 Night plans uploaded on the Cambridge server
-----> 2023-05-29 Zip Plans_by_dates folder uploaded on the Cambridge server
-----> 2023-05-29 Zip Plans_by_dates folder uploaded on the HUB for Io
-----> 2023-05-30 Plans uploaded on the Cambridge server


Permission denied, please try again.
Permission denied, please try again.


-----> 2023-05-30 Night plans uploaded on the Cambridge server
-----> 2023-05-30 Zip Plans_by_dates folder uploaded on the Cambridge server
-----> 2023-05-30 Zip Plans_by_dates folder uploaded on the HUB for Io
-----> 2023-05-31 Plans uploaded on the Cambridge server
-----> 2023-05-31 Night plans uploaded on the Cambridge server
-----> 2023-05-31 Zip Plans_by_dates folder uploaded on the Cambridge server
-----> 2023-05-31 Zip Plans_by_dates folder uploaded on the HUB for Io


kex_exchange_identification: read: Connection reset by peer
lost connection
Permission denied, please try again.
kex_exchange_identification: Connection closed by remote host
lost connection
Permission denied, please try again.
Permission denied, please try again.
Permission denied, please try again.
ssh: connect to host appcs.ra.phy.cam.ac.uk port 22: Connection refused
lost connection
ssh: connect to host appcs.ra.phy.cam.ac.uk port 22: Connection refused
lost connection
ssh_dispatch_run_fatal: Connection to 131.111.48.31 port 22: Operation timed out
lost connection
client_loop: send disconnect: Broken pipe
lost connection
ssh_dispatch_run_fatal: Connection to 131.111.48.31 port 22: Operation timed out
lost connection
client_loop: send disconnect: Broken pipe
lost connection
client_loop: send disconnect: Broken pipe
lost connection
client_loop: send disconnect: Broken pipe
lost connection
client_loop: send disconnect: Broken pipe
lost connection
ssh: connect to host appcs.ra.phy.cam.

In [ ]:
# Cell 15 -- BUTTON -- Make schedule for TS and TN (docx files)

docx_widget = widgets.Button(
    value=False,
    description='Update .docx file',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Docx',
    icon='check'
)
start_date_docx_widget =widgets.DatePicker(
    description='Start .docx',
    disabled=False)

end_date_docx_widget =widgets.DatePicker(
    description='End .docx',
    disabled=False)

def run_save(b): 
    display(Javascript('Jupyter.notebook.execute_cells([9])'))
    
    
docx = widgets.Accordion(children=[widgets.HBox([start_date_docx_widget,end_date_docx_widget,docx_widget])])

display(docx)

docx_widget.on_click(run_save)



In [ ]:
# Cell 16 -- Make schedule for TS and TN (docx files)
schedule = SPOCKLT.Schedules()
schedule.observatory = observatory_widget.value
schedule.telescope = observatory_widget.value
start_date = Time(datetime.datetime.strptime(str(start_date_docx_widget.value), '%Y-%m-%d')+
                  datetime.timedelta(hours=15)).iso
end_date = Time(datetime.datetime.strptime(str(end_date_docx_widget.value), '%Y-%m-%d')+
                datetime.timedelta(hours=15)).iso
schedule.date_range = Time([start_date,end_date])
SPOCKLT.make_docx_schedule(observatory_name=schedule.observatory,telescope=schedule.telescope,
                           date_range=schedule.date_range, name_operator=operator_widget.value)

In [ ]:
# Cell 17 -- Plot phase coverage SPECULOOS targets
@interact
def plt_phase_cov(Target=pd.concat([pd.read_csv(target_list_from_stargate_path,
                                          delimiter=',')['Sp_ID'], 
                                    schedules_st.target_table_spc_follow_up['Sp_ID']]),
                  periods = widgets.FloatRangeSlider(value=[0, 6],min=0,max=20.0,step=0.1,
                                                     description='Period range:', disabled=False,
                                                     continuous_update=True,orientation='horizontal',
                                                     readout=True,readout_format='.1f',)):
    schedule = SPOCKLT.Schedules()
    schedule.observatory_name = "SSO"
    schedule.load_parameters()
    
    try: 
        SPOCKplot.phase_coverage_given_target(target=Target,pmin=periods[0],pmax=periods[1])
    except ValueError:
        print(Fore.RED + 'ERROR:  ' + Fore.BLACK + ' No data for this target ! ')
    

In [ ]:
# Cell 18 -- Read night plans
@interact
def night_block(telescope=['Saint-Ex','Io','Europa','Ganymede','Callisto','Artemis','TS_La_Silla','TN_Oukaimeden'],day_widget = widgets.DatePicker(description='Pick a Date')):
    try:
        day = Time(datetime.datetime.combine(day_widget,datetime.time(15,0,0))).iso
        date = Time(day,out_subfmt='date').iso
        nightp = SPOCKstats.read_night_plans_server(telescope,date)
        if nightp.columns[0] == '<!DOCTYPE':
            print('ERROR: No night plan on this date ! ')
        else: 
            display(nightp)
    except TypeError:
        print('ERROR: Choose a date ! ')


In [ ]:
# Cell 19 -- Show Info targets
@interact
def plt_target(Target=list(pd.read_csv(target_list_from_stargate_path,delimiter=',')['Sp_ID'])):
    target_list = pd.read_csv(target_list_from_stargate_path,sep=',')
    target_list.drop(['fc_url','Unnamed: 42','phot_spt_ref','phot_spt','spec_spt','spec_spt_ref','telescope'],axis='columns',inplace=True)
    a = target_list.where((target_list['Sp_ID']==Target))
    a['Gaia_ID'] = a['Gaia_ID'].map('{:.2f}'.format)
    a  = a.dropna()
    idx_gaia_id = np.where((target_list['Sp_ID']==Target))[0]
    a['Gaia_ID'] = int(target_list["Gaia_ID"][idx_gaia_id])
    display(a)
    #try:
    #    day = Time(datetime.datetime.combine(day_widget,datetime.time(15,0,0)),format='datetime')
    #    SPOCKstats.read_night_plans_server(telescope,date)
#
    #except TypeError:                                                                                
    #    print('ERROR: Choose a date ! ')

In [ ]:
# Cell 20 -- Predict transit windows from WG6
@interact
def predictions(Target=list(schedules_st.target_table_spc_follow_up['Sp_ID']),
                day_widget = widgets.DatePicker(description='Pick a Date'),
                nb_transits=widgets.IntSlider(min=1, max=30, step=1, value=10)):
    if day_widget is not None:
        if Target:
            day = Time(datetime.datetime.combine(day_widget,datetime.time(15,0,0))).iso
            ra,dec,timing,period,duration = SPOCKST.get_info_follow_up_target(Target,
                                                target_list_follow_up=schedules_st.target_table_spc_follow_up)
            table_prediction = SPOCKST.prediction(name=Target,ra=ra,dec=dec,
                               timing=timing,
                               period=period,duration=duration,
                               start_date=day,ntr=nb_transits)
            display(table_prediction)
        else:
            sys.exit('WARNING: If you don\'t select a target you must enter the info in the next tab')
    else:
        pass


In [ ]:
# Cell 21 -- Predict transit windows from ephemeris
day = Time(datetime.datetime.combine(prediction_date_widget.value,datetime.time(15,0,0))).iso
table_prediction = SPOCKST.prediction(name=name_widget.value,ra=ra_text.value,dec=dec_text.value,
                               timing=timing_text.value,
                               period=period_text.value,duration=duration_text.value,
                               start_date=day,ntr=nb_transits_widget.value)
display(table_prediction)

In [ ]:
# CELL 22 -- Plot visibility plot target follow-up
@interact
def plt_target(site=['Saint-Ex','SSO','SNO','TS_La_Silla','TN_Oukaimeden'],\
               target=schedules_st.target_table_spc_follow_up['Sp_ID'],\
               day_widget = widgets.DatePicker(description='Pick a Date')):
    
    schedules_st = SPOCKST.Schedules() 
    schedules_st.load_parameters()
    idx = np.where((schedules_st.target_table_spc_follow_up['Sp_ID'] == target))[0]
    try:
        day = Time(datetime.datetime.combine(day_widget,datetime.time(15,0,0)),format='datetime')
        SPOCKplot.airmass_altitude_plot_nolist(name_observatory=site, day=day, 
                                               target = target, 
                                               ra = schedules_st.target_table_spc_follow_up['RA'][idx], 
                                               dec = schedules_st.target_table_spc_follow_up['DEC'][idx])
        
    except TypeError:
        print('ERROR: Choose a date ! ')


In [ ]:
# CELL 23 -- Plot visibility plot target special
@interact
def plt_target(site=['Saint-Ex','SSO','SNO','TS_La_Silla','TN_Oukaimeden'],\
               target=schedules_st.target_table_spc['Sp_ID'],\
               day_widget = widgets.DatePicker(description='Pick a Date')):
    
    schedules_st = SPOCKST.Schedules() 
    schedules_st.load_parameters()
    idx = np.where((schedules_st.target_table_spc['Sp_ID'] == target))[0]
    try:
        day = Time(datetime.datetime.combine(day_widget,datetime.time(15,0,0)),format='datetime')
        SPOCKplot.airmass_altitude_plot_nolist(name_observatory=site, day=day, 
                                               target = target, 
                                               ra = schedules_st.target_table_spc['RA'][idx], 
                                               dec = schedules_st.target_table_spc['DEC'][idx])
        
    except TypeError:
        print('ERROR: Choose a date ! ')

In [ ]:
# Cell 24 -- Predict transit windows from WG6 for MANGOs candidates
@interact
def predictions_mangos(Target=list(schedules_st.target_table_spc_follow_up['Sp_ID']),
                day_widget = widgets.DatePicker(description='Pick a Date'),
                nb_transits=widgets.IntSlider(min=1, max=30, step=1, value=10)):
    if day_widget is not None:
        if Target:
            day = Time(datetime.datetime.combine(day_widget,datetime.time(15,0,0))).iso
            ra,dec,timing,period,duration = SPOCKST.get_info_follow_up_target(Target,
                                                target_list_follow_up=schedules_st.target_table_spc_follow_up)
            table_prediction = SPOCKST.prediction(name=Target,ra=ra,dec=dec,
                               timing=timing,
                               period=period,duration=duration,
                               start_date=day,ntr=nb_transits)
            display(table_prediction)
        else:
            sys.exit('WARNING: If you don\'t select a target you must enter the info in the next tab')
    else:
        pass
